### Importing the required libraries

In [234]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from pprint import pprint
import json

### Loading the datasets

In [153]:
sales_reciepts = pd.read_csv("datasets/201904 sales reciepts.csv")
sales_reciepts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [154]:
sales_reciepts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49894 entries, 0 to 49893
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    49894 non-null  int64  
 1   transaction_date  49894 non-null  object 
 2   transaction_time  49894 non-null  object 
 3   sales_outlet_id   49894 non-null  int64  
 4   staff_id          49894 non-null  int64  
 5   customer_id       49894 non-null  int64  
 6   instore_yn        49894 non-null  object 
 7   order             49894 non-null  int64  
 8   line_item_id      49894 non-null  int64  
 9   product_id        49894 non-null  int64  
 10  quantity          49894 non-null  int64  
 11  line_item_amount  49894 non-null  float64
 12  unit_price        49894 non-null  float64
 13  promo_item_yn     49894 non-null  object 
dtypes: float64(2), int64(8), object(4)
memory usage: 5.3+ MB


In [155]:
products = pd.read_csv("datasets/product.csv")
products.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


In [156]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   product_id               88 non-null     int64  
 1   product_group            88 non-null     object 
 2   product_category         88 non-null     object 
 3   product_type             88 non-null     object 
 4   product                  88 non-null     object 
 5   product_description      88 non-null     object 
 6   unit_of_measure          88 non-null     object 
 7   current_wholesale_price  88 non-null     float64
 8   current_retail_price     88 non-null     object 
 9   tax_exempt_yn            88 non-null     object 
 10  promo_yn                 88 non-null     object 
 11  new_product_yn           88 non-null     object 
dtypes: float64(1), int64(1), object(10)
memory usage: 8.4+ KB


### Merging both datasets for further preprocessing

In [157]:
selected_sales_reciepts_columns = ["transaction_id", "transaction_date", "sales_outlet_id", "customer_id", "product_id", "quantity"]
selected_products_columns = ["product_id", "product_category", "product"]
dataset = pd.merge(sales_reciepts[selected_sales_reciepts_columns], products[selected_products_columns], "left", "product_id")
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


### Preprocessing the merged dataset

In [158]:
dataset["product"] = dataset["product"].str.replace(" Lg", "")
dataset["product"] = dataset["product"].str.replace(" Rg", "")
dataset["product"] = dataset["product"].str.replace(" Sm", "")

In [159]:
dataset["transaction"] = dataset["transaction_id"].astype(str) + "_" + dataset["customer_id"].astype(str)
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai,7_558
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian,11_781
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea,19_788
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend,32_683
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River,33_99


In [160]:
# choosing only a subset products
selected_products = [
                        'Cappuccino', 'Latte', 'Espresso shot',  \
                        'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                        'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                        'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                        'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                        'Ginger Biscotti'
                    ]

dataset = dataset[dataset["product"].isin(selected_products)]
dataset.head()


,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


In [161]:
# removing the transaction that belongs to not loyal clients
transaction_counts = dataset["transaction"].value_counts()
loyal_clients = transaction_counts[transaction_counts>1].index
dataset = dataset[dataset["transaction"].isin(loyal_clients)]

In [162]:
dataset["product_category"].value_counts()

product_category
Bakery                3800
Coffee                3174
Flavours              2246
Drinking Chocolate     947
Packaged Chocolate      22
Name: count, dtype: int64

In [163]:
dataset["product"].value_counts()

product
Cappuccino                  1290
Latte                       1256
Dark chocolate               969
Chocolate Croissant          636
Espresso shot                628
Sugar Free Vanilla syrup     605
Chocolate syrup              568
Carmel syrup                 561
Hazelnut syrup               512
Ginger Scone                 417
Jumbo Savory Scone           357
Croissant                    355
Chocolate Chip Biscotti      352
Cranberry Scone              350
Almond Croissant             347
Hazelnut Biscotti            338
Oatmeal Scone                334
Ginger Biscotti              314
Name: count, dtype: int64

### Popularity recommendation engine

In [164]:
popularity_recommendation = dataset.groupby(["product", "product_category"]).count().reset_index()
popularity_recommendation = popularity_recommendation[["product", "product_category", "transaction_id"]]
popularity_recommendation = popularity_recommendation.rename(columns={"transaction_id": "number_of_transactions"})
popularity_recommendation

,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,347
1,Cappuccino,Coffee,1290
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,352
4,Chocolate Croissant,Bakery,636
5,Chocolate syrup,Flavours,568
6,Cranberry Scone,Bakery,350
7,Croissant,Bakery,355
8,Dark chocolate,Drinking Chocolate,947
9,Dark chocolate,Packaged Chocolate,22


In [ ]:
popularity_recommendation.to_csv("api/recommendation_objects/popularity_recommendation.csv", index=False)

### Apriori recommnedation engine

In [166]:
train_basket = dataset.groupby(["transaction","product"])["product"].count().reset_index(name="count")
train_basket.head()

,transaction,product,count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1


In [167]:
train_basket = train_basket.pivot_table(index="transaction", columns="product", values="count", aggfunc="sum").fillna(0)
train_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0


In [168]:
def encode_units(x):
    if x >=1:
        return 1
    return 0

train_basket = train_basket.map(encode_units)
train_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [169]:
frequent_items = apriori(train_basket, min_support=0.05, use_colnames=True)
frequent_items.head()

c:\Users\Osamih\Desktop\AI Chatbot\.venv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.115646,(Almond Croissant)
1,0.388889,(Cappuccino)
2,0.191232,(Carmel syrup)
3,0.112623,(Chocolate Chip Biscotti)
4,0.135676,(Chocolate Croissant)


In [170]:
rules_basket = association_rules(frequent_items, metric = "lift", min_threshold=1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cappuccino),(Almond Croissant),0.388889,0.115646,0.053288,0.137026,1.184874,0.008314,1.024775,0.255319
1,(Almond Croissant),(Cappuccino),0.115646,0.388889,0.053288,0.460784,1.184874,0.008314,1.133333,0.176432
2,(Dark chocolate),(Almond Croissant),0.277022,0.115646,0.057445,0.207367,1.793115,0.025409,1.115717,0.611791
3,(Almond Croissant),(Dark chocolate),0.115646,0.277022,0.057445,0.496732,1.793115,0.025409,1.436567,0.500152
4,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,0.010235,1.031236,0.304358


In [185]:
rules_basket[rules_basket["antecedents"]=={"Cappuccino"}].sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
26,(Cappuccino),(Sugar Free Vanilla syrup),0.388889,0.200302,0.113001,0.290573,1.450674,0.035105,1.127245,0.508361
13,(Cappuccino),(Chocolate syrup),0.388889,0.188964,0.109599,0.281827,1.491429,0.036113,1.129304,0.539185
6,(Cappuccino),(Carmel syrup),0.388889,0.191232,0.102797,0.264334,1.382270,0.028429,1.099369,0.452540
22,(Cappuccino),(Hazelnut syrup),0.388889,0.171580,0.090325,0.232264,1.353682,0.023600,1.079044,0.427539
20,(Cappuccino),(Ginger Scone),0.388889,0.133409,0.059713,0.153547,1.150951,0.007832,1.023791,0.214614
16,(Cappuccino),(Croissant),0.388889,0.114135,0.056311,0.144801,1.268685,0.011926,1.035859,0.346553
8,(Cappuccino),(Chocolate Chip Biscotti),0.388889,0.112623,0.056311,0.144801,1.285714,0.012514,1.037626,0.363636
10,(Cappuccino),(Chocolate Croissant),0.388889,0.135676,0.056311,0.144801,1.067250,0.003548,1.010669,0.103112
24,(Cappuccino),(Jumbo Savory Scone),0.388889,0.114890,0.055178,0.141885,1.234962,0.010498,1.031458,0.311333
18,(Cappuccino),(Ginger Biscotti),0.388889,0.106198,0.053666,0.137998,1.299441,0.012367,1.036891,0.377081


In [217]:
product_categories_dict = dataset[["product","product_category"]].drop_duplicates().set_index("product").to_dict()["product_category"]
product_categories_dict

{'Cappuccino': 'Coffee',
 'Jumbo Savory Scone': 'Bakery',
 'Latte': 'Coffee',
 'Chocolate Chip Biscotti': 'Bakery',
 'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Chocolate Croissant': 'Bakery',
 'Dark chocolate': 'Packaged Chocolate',
 'Cranberry Scone': 'Bakery',
 'Croissant': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Ginger Biscotti': 'Bakery',
 'Oatmeal Scone': 'Bakery',
 'Ginger Scone': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours',
 'Sugar Free Vanilla syrup': 'Flavours'}

In [235]:
recommendations_json = {}
antecedents = rules_basket["antecedents"].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket["antecedents"] == antecedent]
    df_rec = df_rec.sort_values("confidence", ascending=False)

    antecedent = "_".join(antecedent)
    recommendations_json[antecedent] = []
    for _, row in df_rec.iterrows():
        consequents = row["consequents"]
        for consequent in consequents:
            alread_exists = False
            for x in recommendations_json[antecedent]:
                if consequent ==  x["product"]:
                    alread_exists = True
            
            if alread_exists:
                continue
            recommendations_json[antecedent].append(
                {
                    "product": antecedent,
                    "product_category": product_categories_dict[antecedent],
                    "confidence": row["confidence"]
                }
            )
pprint(recommendations_json)
    

{'Almond Croissant': [{'confidence': 0.4967320261437908,
                       'product': 'Almond Croissant',
                       'product_category': 'Bakery'},
                      {'confidence': 0.47058823529411764,
                       'product': 'Almond Croissant',
                       'product_category': 'Bakery'},
                      {'confidence': 0.46078431372549017,
                       'product': 'Almond Croissant',
                       'product_category': 'Bakery'}],
 'Cappuccino': [{'confidence': 0.29057337220602525,
                 'product': 'Cappuccino',
                 'product_category': 'Coffee'},
                {'confidence': 0.2818270165208941,
                 'product': 'Cappuccino',
                 'product_category': 'Coffee'},
                {'confidence': 0.26433430515063167,
                 'product': 'Cappuccino',
                 'product_category': 'Coffee'},
                {'confidence': 0.23226433430515062,
                 'product

In [ ]:
with open("api/recommendation_objects/apriori_recommendation.json", "w") as json_file:
    json.dump(recommendations_json, json_file)